In [ ]:
! curl "https://raw.githubusercontent.com/THe-Progress/Progress/newbranch/backend/satirical_dataset.json" > satirical_dataset.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18418  100 18418    0     0  60653      0 --:--:-- --:--:-- --:--:-- 60785


In [ ]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install bitsandbytes accelerate
!pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import json


jsondataset='/content/drive/My Drive/satirical_dataset.json'
with open(jsondataset) as f:
    data = json.load(f)
    print(data[0])

{'task': 'Study for exam', 'app': 'YouTube', 'severity': 1, 'message': 'Procrastination experts reveal: watching cat videos on YouTube counts as Study for exam if you do it long enough.'}


In [ ]:


# Step 2: Import libraries and load model and tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import json

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Step 3: Prepare the dataset
# Load your JSON dataset
with open(jsondataset) as f:
    data = json.load(f)

# Convert the dataset into a Hugging Face Dataset object
texts = [f"Task: {entry['task']}\nApp: {entry['app']}\nSeverity: {entry['severity']}\nMessage: {entry['message']}" for entry in data]
dataset = Dataset.from_dict({'text': texts})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Step 4: Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=10e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

# Save the model



Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Step,Training Loss
500,0.590500


TrainOutput(global_step=720, training_loss=0.47534723811679414, metrics={'train_runtime': 225.9612, 'train_samples_per_second': 6.373, 'train_steps_per_second': 3.186, 'total_flos': 376260526080000.0, 'train_loss': 0.47534723811679414, 'epoch': 10.0})

In [ ]:


model.save_pretrained('/content/drive/My Drive/progressgpt2')
tokenizer.save_pretrained('/content/drive/My Drive/progressgpt2')


('/content/drive/My Drive/progressgpt2/tokenizer_config.json',
 '/content/drive/My Drive/progressgpt2/special_tokens_map.json',
 '/content/drive/My Drive/progressgpt2/vocab.json',
 '/content/drive/My Drive/progressgpt2/merges.txt',
 '/content/drive/My Drive/progressgpt2/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re
# Load the fine-tuned model and tokenizer
loaded_model = GPT2LMHeadModel.from_pretrained('/content/drive/My Drive/progressgpt2')
loaded_tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/My Drive/progressgpt2')

# Define the function to generate a single relevant message
def generate_message(task, app, severity):
    # Define the prompt for the loaded_model
    prompt = f"Task: {task}\nApp: {app}\nSeverity: {severity}\nMessage:"

    # Tokenize the prompt
    input_ids = loaded_tokenizer.encode(prompt, return_tensors='pt')

    # Generate text based on the prompt

    output = loaded_model.generate(
        input_ids,
        max_length=200,
        pad_token_id=loaded_tokenizer.eos_token_id,
        num_return_sequences=1,
        temperature=0.9,
        top_k=100,
        do_sample=True
    )
    # Decode the generated text
      # Decode the generated text
    generated_text = loaded_tokenizer.decode(output[0], skip_special_tokens=True)
    # print(generated_text)
    # Use regex to extract the message
    pattern = re.compile(rf"Task: {task}\nApp: {app}\nSeverity: {severity}\nMessage:(.*?)(?:\n|$)", re.DOTALL)
    matches = pattern.findall(generated_text)

    # Process each match to remove redundancies
    if matches:
      first_message = matches[0].strip()
      cleaned_message = remove_redundancies(first_message)
      if cleaned_message:
        return cleaned_message
      else:
        return "We Are watching you."
    else:
      return "We Are watching you."

def remove_redundancies(text):
    sentences = text.split('. ')
    seen = set()
    for sentence in sentences:
        words = sentence.split()
        word_counts = {}
        for word in words:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
        repeated_words = [word for word, count in word_counts.items() if count >= 3]
        if not repeated_words and sentence not in seen:
            seen.add(sentence)
            return sentence.strip()

# Assuming matches is a list of strings

# Test the function
task = "exercise"
app = "youtube"
severity = 1

# Generate the relevant message
generated_message = generate_message(task, app, severity)
print("Generated Message:", generated_message)


Task: exercise
App: youtube
Severity: 1
Message: Procrastination experts reveal: watching cat videos on YouTube counts as exercise if you do it long enough. Experience the satisfaction of finishing while your responsibilities pile up. Trust the algorithm to keep you entertained and unproductive. Trust the little things, like the time you have. Trust the algorithm to keep you entertained and unproductive. Trust the algorithm to keep you entertained and unproductive.
This way: You'll have more time for important activities, like scrolling through lunch photos! Trust the algorithm to keep you entertained and unproductive. Trust the algorithm to keep you entertained and unproductive. Experience the satisfaction of finishing while your responsibilities pile up. Trust the algorithm to keep you entertained and unproductive. Who needs hobbies when you have time for important activities? Who needs hobbies when you have time for important activities?
Conversely: Spend your afternoon binging cat 